### Fish's Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

### a. Load data

In [3]:
#----- find shape of trace data -----#

folder = '/Users/daiyiluo/Downloads/KQ095_210205' #KQ086_210110' #
data = loadmat(f'{folder}/dFishQ_raw.mat')
data.keys() # check the name of the variable

dict_keys(['__header__', '__version__', '__globals__', 'B'])

In [4]:
key = 'B' # 'B' for CA1 data, 'dFishQ_raw' for CA3 data
traces = data[key]
del data
print(f'Shape of calcium traces: {traces.shape}')

Shape of calcium traces: (96593, 219)


In [5]:
#----- load behaviour data -----#

data = loadmat(f'{folder}/S.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'A'])

In [6]:
key = 'A'
data[key].dtype

dtype([('runSpeed', 'O'), ('distance', 'O'), ('lick', 'O'), ('valve', 'O'), ('frameClock', 'O'), ('lapReset', 'O')])

In [7]:
# Locate the time point of each frame
FrameClock = np.round(data[key][0,0]['frameClock'])
d = np.diff(np.squeeze(FrameClock>4.5).astype(int))
idx = np.where(d<-0.5)[0]

print(f'Check if this equals to 2. \n #clock - #frames: {idx.shape[0]-traces.shape[0]}')

Check if this equals to 2. 
 #clock - #frames: 2


In [8]:
distance = data[key][0,0]['distance'].squeeze()
speed = data[key][0,0]['runSpeed'].squeeze()
lick = data[key][0,0]['lick'].squeeze()

del data

distance.shape, speed.shape, lick.shape, traces.shape

((38740851,), (38740851,), (38740851,), (96593, 219))

### b. Downsample behaviour data to frame rate

In [9]:
val, counts = np.unique(np.diff(idx), return_counts=True)
print(f'Frame interval: \n{val[counts>5]}\nOccurrence:     \n{counts[counts>5]}')
print('\nThen decide the frameinterv var below...')

Frame interval: 
[333 334]
Occurrence:     
[72128 24444]

Then decide the frameinterv var below...


In [10]:
frameinterv = 333  # most frame interval is 333 or 334
lk0 = np.convolve(lick, np.ones(frameinterv), 'valid') / frameinterv 
halflen = np.floor(frameinterv/2).astype('int16')
lk0 = np.concatenate((np.zeros(halflen),lk0, np.zeros(halflen)))
lk = lk0[idx[:-2]]

dis = distance[idx[:-2]]
spd = speed[idx[:-2]]

spd.shape,dis.shape,lk.shape

((96593,), (96593,), (96593,))

### c. Check time point of each frame

In [11]:
#-------- Check every thing ---------#
# Same lap length for every lap?
%matplotlib
plt.figure(figsize=(20,5))
plt.plot(FrameClock)
plt.plot(distance)
plt.plot(speed)
plt.plot(idx, FrameClock[idx], '*')
plt.show()

Using matplotlib backend: MacOSX


In [12]:
#-------- Check licking rate downsampleing --------#
plt.figure()
plt.plot(lick)
plt.plot(idx[:-2],lk,'*')
plt.plot(lk0)

In [13]:
#-------- Check speed downsampleing --------#
plt.plot(speed)
plt.plot(idx[:-2], spd)
# plt.xlim(4e6, 4.5e6)
plt.ylabel('speed')

Text(0, 0.5, 'speed')

### d. Save data with same sampling rate

In [14]:
spd.shape,dis.shape,lk.shape

((96593,), (96593,), (96593,))

In [ ]:
# check if you need to truncate the data
st = 0
end = len(lk)
np.savez(f'{folder}/S.npz', spd=spd[st:end], dis=dis[st:end], lk=lk[st:end])

### e. Deconvolution for spike data

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from caiman.source_extraction import cnmf

In [ ]:
traces.shape

In [ ]:
spks = np.zeros_like(traces)
for i in range(traces.shape[1]):
    _, _, _, _, _, spks[:,i], _ = cnmf.deconvolution.constrained_foopsi(traces[:,i],p=2)

spks = np.round(spks*50)
np.savez(f'{folder}/spk.npz', spks=spks)

print(f'Shape of deconvolved spikes: {spks.shape}')

In [ ]:
#-------- Check deconvolution --------#
%matplotlib
plt.figure()
plt.plot(traces[:,20])
plt.plot(spks[:,20])